In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 23 20:27:23 2022

@author: geng8
"""

import pandas as pd
import os
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk as nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

import string
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk.tokenize import word_tokenize
from sklearn.cluster import KMeans, DBSCAN
from wordcloud import WordCloud
from wordcloud import STOPWORDS
import matplotlib.pyplot as plt 
from collections import defaultdict
from efficient_apriori import apriori
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kovalsky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kovalsky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#preprocess our data
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
#our data have a special punctuation '—' , need to be removed.
exclude.add('—')
exclude.add('“')
exclude.add('”')

In [3]:
#initialize word normalizer and data loader
lemma = WordNetLemmatizer()

data = pd.read_csv("/home/kovalsky/DTU_CTDS/data/articles1.csv", sep=',')

data_content = data["content"]

In [4]:
#clean the content, remove stop words,punctuatio from the content
# and normalize each word
def clean(text):
    """
    This function takes as input a text on which several 
    NLTK algorithms will be applied in order to preprocess it
    """
  
    tokens = word_tokenize(text)
    # Remove the punctuations
    tokens = [word for word in tokens if word not in stop and len(word) > 2]
    # Lower the tokens
    tokens = [word.lower() for word in tokens]
    # Remove stopword
    tokens = [word for word in tokens if not word in stop]
    
    return tokens

In [5]:
#replace the original content with cleaned content 
for i in data_content.index:  
    cleaned = clean(data_content[i])   
    data_content.loc[i] = str([' '.join(cleaned)])

/tmp/ipykernel_6729/3726915952.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_content.loc[i] = str([' '.join(cleaned)])


In [ ]:
#Count Vectoriser then tidf transformer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data_content)
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(X)

In [ ]:
num_clusters = 3
km = KMeans(num_clusters=num_clusters)
km.fit(tfidf)
print(km.labels_)
clusters = km.labels_.tolist()

[1 0 2 ... 1 0 0]


In [ ]:
cluster0 = []
cluster1 = []
cluster2 = []

for i in clusters:
    if clusters[i] == 0:
        cluster0.append(data_content[i])

    if clusters[i] == 1:
        cluster1.append(data_content[i])

    if clusters[i] == 2:
        cluster2.append(data_content[i])